# Improving Neural Net accuracy with Batch Normalization

In [1]:
%pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import keras_tuner as kt
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from pathlib import Path

## Load CIFARS 10 dataset

In [3]:
cifar = tf.keras.datasets.cifar10.load_data()

(x_train, y_train), (x_test, y_test) = cifar

# Scale down pixel intensities
x_train, x_test = x_train / 255.0, x_test / 255.0

print(x_train.shape) # Look at data

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
(50000, 32, 32, 3)


## Create Initial Model

In [4]:
def build_model(hp):
    n_hidden = hp.Int("n_hidden", min_value=0, max_value=20, default=2)
    n_neurons = hp.Int("n_neurons", min_value=16, max_value=512)
    learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=5e-1, sampling="log")
    optimizer = hp.Choice("optimizer", values=["sgd", "adam", "nadam"])

    if optimizer == "sgd":
      optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == "adam":
      optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    else:
      optimizer = tf.keras.optimizers.Nadam(learning_rate=learning_rate)

    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Flatten(input_shape=(32, 32, 3)))

    for _ in range(n_hidden):
      model.add(tf.keras.layers.Dense(n_neurons, use_bias=False))
      model.add(tf.keras.layers.BatchNormalization()) # Use batch normalization in model (activation function after normalizatiom
      model.add(tf.keras.layers.Activation("swish"))

    model.add(tf.keras.layers.Dense(10, activation="softmax"))

    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    return model

class HyperModel(kt.HyperModel):
  def build(self, hp):
    return build_model(hp)

  def fit(self, hp, model, X, y, **kwargs):
    if hp.Boolean("normalize"):
      norm_layer = tf.keras.layers.Normalization()
      X = norm_layer(X)
    return model.fit(X, y, **kwargs)

## Train Model

### Do Hyperparameter tuning

In [5]:
hyperband_tuner = kt.Hyperband(
    HyperModel(),
    objective="val_accuracy",
    max_epochs=10,
    factor=3,
    directory="cifar_10_hyperband",
    project_name="hyperband_tuning")

/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True) # Interrupts training when no progress on validation set for 2 trials
model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint("cifar10_model.keras",
                                                         save_best_only=True) # Auto save best model

callbacks = [early_stopping_cb, model_checkpoint_cb]


hyperband_tuner.search(x_train, y_train, epochs=10, validation_split=0.2,
                             callbacks=callbacks)

Trial 30 Complete [00h 01m 19s]
val_accuracy: 0.4659999907016754

Best val_accuracy So Far: 0.4975000023841858
Total elapsed time: 00h 20m 48s


### Get best model

In [7]:
top_3_models = hyperband_tuner.get_best_models(num_models=3)
best_model = top_3_models[0]

best_trial = hyperband_tuner.oracle.get_best_trials(num_trials=1)[0]
best_trial.summary()

Trial 0025 summary
Hyperparameters:
n_hidden: 12
n_neurons: 379
learning_rate: 0.0007097566850047709
optimizer: nadam
normalize: True
tuner/epochs: 10
tuner/initial_epoch: 4
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0021
Score: 0.4975000023841858


## Evaluate Model

In [8]:
best_model.evaluate(x_test, y_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.4903 - loss: 1.4387


[1.4377515316009521, 0.492000013589859]